In [7]:
import joblib
import pickle
import sklearn

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import dalex as dx

from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier

pd.options.plotting.backend = 'plotly'
import plotly

In [9]:
sklearn.__version__

'0.24.1'

In [15]:
!python --version

Python 3.8.2


In [4]:
def custom_residual_function(model, X, y):
    probs = model.predict_proba(X)
    real_class_pred = np.array([probs[i][val] for i, val in enumerate(y)])
    return np.abs((1 - real_class_pred).astype('float64'))

def custom_predict(model, X):
    return model.predict_proba(X)

def custom_predict_old(model, X):
    pred_class = np.argmax(model.predict_proba(X), axis=1)
    return pred_class.astype('float')

def loss_accuracy(y_real, y_pred):
    return 1 - accuracy_score(y_real, y_pred)

def loss_multiclass(y_real, y_proba):
    proba_real = np.array([el[y_real[i]] for i, el in enumerate(y_proba)])
    return np.mean(1 - proba_real)

In [14]:
model = joblib.load('Models/GB/Base_corr_Andi_2D/model.sav')

KeyError: 10